# Final project of the course Machine Learning with Python 2021

For the final project, it is asked to create a client list. There is a data set with information over the profit and damage caused by 5000 russian customers. Then there is another data set with a list of applicants. The goal is to create a list of 200 customers that offers a balance between the anticipated damages and profit. There is a recommended list of 5 steps that could be used (but it is not obligatory to follow), to have the list. A minimal solution was also offered to serve as a guide and to reduce ambiguity. 

Possible steps:
1) prepare the data set

    • briefly survey the data
    • deal with data issues:
    • appropriately handle categorical data
    • treat missing data
    • identify outliers, and choose whether or not to make your analysis more robust by removing these.

2) predict the projected revenue per clients

    • choose an algorithm, and train it in an optimal way
    • score the 500 applicants

3) predict which clients will cause damage

    • choose an algorithm, and train it in an optimal way
    • score the 500 applicants

4) for those that will wreak havoc, predict the amount of damage they will cause

    • choose an algorithm, and train it in an optimal way
    • score the 500 applicants

5) create a measure of the expected value of each applicant, and create an optimal selection of 200 guests

For the development of the project it was decided to use the proposed steps and and use the solution as a guide, to build upon it. Also, as the team was working on the code at the same time, a Github repository was created to facilitate this. The first step that was carried out was Preprocessing the data.

## 1. Preprocessing de data

First of all, the libraries need to be imported. 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import animation as ani, pyplot as plt
import seaborn as sns #pretty graphics R style

from IPython.display import HTML
plt.style.use('seaborn-darkgrid')
from sklearn.inspection import permutation_importance
import matplotlib as mpl 
import matplotlib.pyplot as plt #graphics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler #library for the rescaling
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from treeinterpreter import treeinterpreter as ti, utils
import joblib
import getpass
from pathlib import Path

To avoid changing the directory everytime a different team member ran the code, the following was created:

In [2]:
if getpass.getuser() == 'daniel':
    project_root_path = Path("//home/daniel/Projects/Python ML Pipeline")
    data_raw_path = project_root_path / 'data' / 'raw'
    data_output_path = project_root_path / 'data' / 'processed'
    # Lets read the training dataset
    data_train = pd.read_csv(data_raw_path / 'train_V2.csv')
    # Now we read the data set of the potential clients
    score = pd.read_csv(data_raw_path /  'score.csv')
    # We read the dictonary
    dict_features = pd.read_csv(data_raw_path /  'dictionary.csv', delimiter=';', header=None)
    
elif getpass.getuser() == 'maart':
    data_file_path = "C:/Users/maart/Machine Learning/ML-in-Python/data/raw/"
    # Lets read the trining dataset
    data_train = pd.read_csv(data_file_path+'train_V2.csv')
    # Now we read the training data set
    score = pd.read_csv(data_file_path+'score.csv')
    dict_features_path = data_file_path+'dictionary.csv'
    dict_features = pd.read_csv(dict_features_path, delimiter=';', header=None)
    
else:
    project_root_path = Path("C:/Users/mhinojosalee/Documents/GitHub/ML-in-Python")
    data_raw_path = project_root_path / 'data' / 'raw'
    data_output_path = project_root_path / 'data' / 'processed'
    # Lets read the training dataset
    data_train = pd.read_csv(data_raw_path / 'train_V2.csv')
    # Now we read the data set of the potential clients
    score = pd.read_csv(data_raw_path /  'score.csv')
    # We read the dictonary
    dict_features = pd.read_csv(data_raw_path /  'dictionary.csv', delimiter=';', header=None)
   

Some visualization tasks were carried out to get more acquintence with the data. This information can be consulted in====>>>>>

Now, lets show the data set columns as it is, with their columns.

In [3]:
data_train.shape
pd.options.display.max_columns = None
data_train.head()


,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,spa_ic,empl_ic,cab_requests,married_cd,bar_no,sport_ic,neighbor_income,age,marketing_permit,urban_ic,dining_ic,presidential,client_segment,sect_empl,prev_stay,prev_all_in_stay,divorce,fam_adult_size,children_no,tenure_mts,tenure_yrs,company_ic,claims_no,claims_am,nights_booked,gender,shop_am,shop_use,retired,gold_status,score1_pos,score1_neg,score2_pos,score2_neg,score3_pos,score3_neg,score4_pos,score4_neg,score5_pos,score5_neg,outcome_profit,outcome_damage_inc,outcome_damage_amount
0,227.0,0.0,3201.0,888.0,6.0,15000.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,True,2.0,1.0,28936.0,37.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0,2.0,476.0,40.0,0.0,0.0,0.0,209.0,M,0.000000,0.0,0.0,0.0,0.467768,0.98334,NaN,NaN,NaN,NaN,0.838147,0.082288,NaN,NaN,1791.66,0,0.00
1,268.0,16.0,1682.0,0.0,0.0,750.0,0.0,0.0,0.0,1.0,1.0,0.0,7.0,True,3.0,0.0,16674.0,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,27.0,2.0,0.0,0.0,0.0,4.0,M,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.955259,1672.78,1,829.66
2,283.0,23.0,1673.0,0.0,0.0,750.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,True,4.0,0.0,32552.0,21.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,95.0,8.0,0.0,0.0,0.0,6.0,M,0.000000,0.0,0.0,0.0,NaN,NaN,0.232375,0.099529,NaN,NaN,NaN,NaN,0.101955,1.743020,1001.40,0,0.00
3,227.0,0.0,1685.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,True,8.0,1.0,32252.0,37.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,2.0,NaN,NaN,0.0,0.0,0.0,4.0,V,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.889793,NaN,NaN,NaN,NaN,1785.59,0,0.00
4,4091.0,1028.0,3425.0,785.0,2.0,14000.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,False,2.0,1.0,29605.0,26.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,354.0,30.0,0.0,0.0,0.0,3.0,V,1454.210627,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.330503,0.766294,0.490486,0.542445,NaN,NaN,3140.74,0,0.00


In the next part, Describe() will be used. This allows to see some statistical details from each column in pandas (pandas is a tool for the manipulation of data).  This includes the mean, the standard deviation, quartiles, min, max.

In [4]:
data_train.describe()

,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,spa_ic,empl_ic,cab_requests,bar_no,sport_ic,neighbor_income,age,marketing_permit,urban_ic,dining_ic,presidential,client_segment,sect_empl,prev_stay,prev_all_in_stay,divorce,fam_adult_size,children_no,tenure_mts,tenure_yrs,company_ic,claims_no,claims_am,nights_booked,shop_am,shop_use,retired,gold_status,score1_pos,score1_neg,score2_pos,score2_neg,score3_pos,score3_neg,score4_pos,score4_neg,score5_pos,score5_neg,outcome_profit,outcome_damage_inc,outcome_damage_amount
count,4947.000000,4947.000000,4947.000000,4954.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4970.000000,4999.000000,4912.000000,4947.000000,4947.000000,4761.000000,4947.000000,4947.000000,4947.000000,4912.000000,4912.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4608.000000,4608.000000,4947.000000,4947.000000,4973.000000,4947.000000,4947.000000,4912.000000,4947.000000,4947.000000,1.225000e+03,1.314000e+03,1.209000e+03,1.304000e+03,1.261000e+03,1.367000e+03,1.223000e+03,1.324000e+03,1.232000e+03,1493.000000,5000.000000,5000.000000,5000.000000
mean,2281.260158,696.057712,3637.900950,145.952967,0.352335,3298.716394,0.041237,0.024661,0.094199,0.390944,0.401811,0.024205,6.051507,5.646250,0.287043,32778.558916,44.901152,0.495452,0.883970,0.049267,0.004275,1.298565,0.213463,0.889832,0.252678,0.102486,1.960986,0.385082,273.111545,22.780165,0.018597,0.218314,121.078826,28.992521,403.019960,0.151873,0.182131,0.034769,4.997356e-01,5.003663e-01,4.985522e-01,4.967340e-01,4.942801e-01,4.985876e-01,4.962065e-01,5.013962e-01,5.009593e-01,5.192953,1967.310930,0.255400,189.970736
std,8365.254507,3051.119275,5726.625669,581.068095,0.889449,4549.646039,0.198858,0.155107,0.292134,0.488011,0.490313,0.153700,3.112104,5.052513,0.452427,6858.671948,16.225094,0.500030,0.320293,0.216447,0.065252,0.800831,0.826006,0.313130,0.434592,0.303317,0.805545,0.832933,152.498416,12.719429,0.135111,0.712408,1783.146726,37.480510,1335.935144,0.358934,0.385991,0.183212,2.879255e-01,2.887168e-01,2.877572e-01,2.897994e-01,2.899165e-01,2.877292e-01,2.886538e-01,2.876226e-01,2.901323e-01,3.159868,1371.061266,0.436129,379.005941
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,-7.871775,10.680000,0.000000,0.000000
25%,229.000000,0.000000,1638.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000,28630.000000,31.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,154.000000,13.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,2.520205e-01,2.510338e-01,2.521282e-01,2.454209e-01,2.405574e-01,2.495061e-01,2.474100e-01,2.506703e-01,2.514905e-01,3.124958,1333.320000,0.000000,0.000000
50%,469.000000,52.000000,1889.000000,0.000000,0.000000,1500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,5.000000,0.000000,31990.000000,45.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,271.000000,23.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,4.974162e-01,4.986215e-01,4.987791e-01,4.985832e-01,4.942465e-01,5.016458e-01,4.933486e-01,5.020603e-01,5.029121e-01,5.188006,1721.235000,0.000000,0.000000
75%,1688.000000,810.000000,3165.500000,0.000000,0.000000,5000.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,9.000000,8.000000,1.000000,35924.000000,57.000000,1.000000,1.00

The next part was done just to see what happened if we deleted the missing values. It was left just for information but it was decided to not just drop missing values.

In [5]:
# total = data_train.isnull().sum().sort_values(ascending=False)
# percent = (data_train.isnull().sum()/data_train.isnull().count()).sort_values(ascending=False)
# (data_train.isnull().sum(axis=1))[data_train.isnull().sum(axis=1) > 30]
# table
# missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# print(missing_data.head(30))
# print(data_train['score2_pos'].value_counts())
# data_train = data_train.drop((missing_data[missing_data['Percent'] > 0.30]).index,1)
# data_train.dropna(inplace=True) 
#we could drop all that is NaN, but we will loose observations. (4425, 43) instead of (4425, 43) 

After this, the preprocessing will actually start. But first, it is important to understand that what is done in the training data set, has to also be done also in the score data set. For this, both data set will be fused, but the outcomes need to be dropped.

In [6]:
print(data_train.columns)
data_feat = data_train.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'], axis=1)
print(data_feat.shape)

Index(['income_am', 'profit_last_am', 'profit_am', 'damage_am', 'damage_inc',
       'crd_lim_rec', 'credit_use_ic', 'gluten_ic', 'lactose_ic',
       'insurance_ic', 'spa_ic', 'empl_ic', 'cab_requests', 'married_cd',
       'bar_no', 'sport_ic', 'neighbor_income', 'age', 'marketing_permit',
       'urban_ic', 'dining_ic', 'presidential', 'client_segment', 'sect_empl',
       'prev_stay', 'prev_all_in_stay', 'divorce', 'fam_adult_size',
       'children_no', 'tenure_mts', 'tenure_yrs', 'company_ic', 'claims_no',
       'claims_am', 'nights_booked', 'gender', 'shop_am', 'shop_use',
       'retired', 'gold_status', 'score1_pos', 'score1_neg', 'score2_pos',
       'score2_neg', 'score3_pos', 'score3_neg', 'score4_pos', 'score4_neg',
       'score5_pos', 'score5_neg', 'outcome_profit', 'outcome_damage_inc',
       'outcome_damage_amount'],
      dtype='object')
(5000, 50)


Then categorical features were chosen. In the next part, the different values of each of them are counted.

In [7]:
print(score.shape)
datafull = pd.concat([data_feat, score])
print(datafull.shape)
print(datafull['client_segment'].value_counts())
print(datafull['sect_empl'].value_counts())
print(datafull['gender'].value_counts())
print(datafull['retired'].value_counts())
print(datafull['gold_status'].value_counts())
print(datafull['prev_stay'].value_counts())
print(datafull['divorce'].value_counts())
print(datafull['married_cd'].value_counts())

(500, 50)
(5500, 50)
1.0    3712
2.0     925
0.0     352
3.0     329
4.0      87
5.0      38
Name: client_segment, dtype: int64
0.0    4820
1.0     468
6.0      78
2.0      45
4.0      29
3.0       3
Name: sect_empl, dtype: int64
M    2734
V    2709
Name: gender, dtype: int64
0.0    4456
1.0     987
Name: retired, dtype: int64
0.0    5241
1.0     202
Name: gold_status, dtype: int64
1.0    4848
0.0     595
Name: prev_stay, dtype: int64
0.0    4884
1.0     559
Name: divorce, dtype: int64
True     4460
False    1040
Name: married_cd, dtype: int64


For the variables, a mode imputation was done instead of making a separate missing category. A disadvantage is that the most frequent value will be favored. But every technique, has its downfalls.

In [8]:
# datafull['client_segment'] = pd.Categorical(datafull['client_segment'])
# datafull['sect_empl'] = pd.Categorical(datafull['sect_empl'])
# datafull['retired'] = pd.Categorical(datafull['retired'])
# datafull['gold_status'] = pd.Categorical(datafull['gold_status'])
# datafull['prev_stay'] = pd.Categorical(datafull['prev_stay'])
# datafull['divorce'] = pd.Categorical(datafull['divorce'])

impute_mode = SimpleImputer (strategy='most_frequent')
for cols in ['client_segment', "credit_use_ic", "gluten_ic", "lactose_ic","insurance_ic","marketing_permit", "presidential", "urban_ic", "prev_all_in_stay", "shop_use", 
             "company_ic", "dining_ic", "spa_ic","sport_ic","empl_ic",'sect_empl', "retired", "gold_status", "prev_stay", 'divorce', "gender"]:  
      datafull[cols] = impute_mode.fit_transform(datafull[[cols]])

The categorical features that had values different from 0 and 1, were dummified (separated in categories) 

In [9]:
datafull['client_segment'] = pd.Categorical(datafull['client_segment'])
datafull['sect_empl'] = pd.Categorical(datafull['sect_empl'])
# The NaN categorie won't be necessary anymore, thanks to the mode imputing.
pd.get_dummies(datafull[['client_segment', 'sect_empl']], dummy_na=False).head()
print(datafull.shape)
datafull2 = pd.concat([datafull,pd.get_dummies(datafull[['gender','client_segment', 'sect_empl']], dummy_na=False)], axis=1)
print(datafull2.shape)
print(datafull2.head(1000))     

(5500, 50)
(5500, 64)
     income_am  profit_last_am  profit_am  damage_am  damage_inc  crd_lim_rec  \
0        227.0             0.0     3201.0      888.0         6.0      15000.0   
1        268.0            16.0     1682.0        0.0         0.0        750.0   
2        283.0            23.0     1673.0        0.0         0.0        750.0   
3        227.0             0.0     1685.0        0.0         0.0          0.0   
4       4091.0          1028.0     3425.0      785.0         2.0      14000.0   
..         ...             ...        ...        ...         ...          ...   
995     3103.0             0.0     9466.0     1206.0         2.0      12500.0   
996        NaN             NaN        NaN        NaN         NaN          NaN   
997      250.0           823.0     1646.0        0.0         0.0       1500.0   
998     6382.0           561.0     7265.0        0.0         0.0       1500.0   
999     5556.0          2464.0     2464.0        0.0         0.0          0.0   

     

After creating the dummies for the necessary variables, the original features and 1 dummy category was dropped in each of them. Also a new  feature, "profit per night" was created.

In [10]:
datafull2.drop(['client_segment', 'sect_empl', 'gender', 'client_segment_5.0','sect_empl_6.0','gender_V'], axis=1, inplace=True)
datafull2.shape

datafull2['profitpernight'] = datafull2['profit_am'] / datafull2['nights_booked']

Now the data with over 25% missing values will be found. Most of it consist in the features related to scores. The solution dropped them, but we decided to try another option.

In [11]:
(datafull2.isnull().mean())[datafull2.isnull().mean() > 0.25]

score1_pos    0.755091
score1_neg    0.736727
score2_pos    0.760000
score2_neg    0.740364
score3_pos    0.746000
score3_neg    0.725091
score4_pos    0.756000
score4_neg    0.736364
score5_pos    0.752000
score5_neg    0.700545
dtype: float64

Something important that was mentioned during class was that sometimes it is not worthy to use very complex methods like soft imputing to avoid dropping features with missing values. So we decided to try something simple: a mean imputing. The scores are quantitative, and of course, a downside of this is that it could bring us distortion.

In [12]:
impute_quant = SimpleImputer (strategy='mean')
for cols in ['score1_pos', 'score1_neg', 'score2_pos', 'score2_neg', 'score3_pos',
       'score3_neg', 'score4_pos', 'score4_neg', 'score5_pos', 'score5_neg']:  # Missing data, Scores are quantitative
      datafull2[cols] = impute_quant.fit_transform(datafull2[[cols]])

Then the missing values per row were dropped (observations). But there were not observations with this percentage of missing values.

In [13]:
print(datafull2.shape)
datafull2.dropna(thresh = datafull2.shape[1]*0.25, axis = 0, inplace = True)
print(datafull2.shape)


(5500, 59)
(5500, 59)


After this, a mean imputation was done for the other missing values that could be present (here the binary data was already treated before). And at the end there was not more missing data.

In [14]:
print(datafull2.isnull().sum().sum())
datafull2.fillna(datafull2.mean(), inplace=True)
print(datafull2.isnull().sum().sum())

2033
0


In the next section, the features will be rescaled.

In [15]:
scaler = StandardScaler()
datafull3 = pd.DataFrame(scaler.fit_transform(datafull2))
datafull3.columns = datafull2.columns


Then, the score and the train datasets will be separated and the outcome features will be reinstated.

In [16]:
data_train = pd.concat([data_train[['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount']],datafull3[0:5000]], axis=1)
print(data_train.shape)
score = datafull3[5000:5500] #The score dataset will be the last 500 observations
score.shape

(5000, 62)


(500, 59)

## 2. Profit Model using GAM

After the preprocessing, the data was split in train and test. The test size was decided to be 20%, so that more data was available for the training.

In [17]:
data_train.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'],1),
                                                    data_train['outcome_profit'], test_size=0.2, random_state=48)
data_train.shape

(5000, 62)

We astart working with splines, to set a baseline quality. In this case instead of using the most important features, we decided to use all the features. 

In [18]:
from pygam import GAM, s, te

GAM(s(0, n_splines=200) + s(1, n_splines=200) + s(2, n_splines=200) + s(3, n_splines=200) + s(4, n_splines=200) + 
    s(5, n_splines=200) + s(6, n_splines=200) + s(7, n_splines=200) + s(8, n_splines=200) + s(9, n_splines=200) + 
    s(10, n_splines=200) + s(11, n_splines=200) + s(12, n_splines=200) + s(13, n_splines=200) + s(14, n_splines=200) + 
    s(15, n_splines=200) + s(16, n_splines=200) + s(17, n_splines=200) + s(18, n_splines=200) + s(19, n_splines=200) + 
    s(20, n_splines=200) + s(21, n_splines=200) + s(22, n_splines=200) + s(23, n_splines=200) + s(24, n_splines=200) + 
    s(25, n_splines=200) + s(26, n_splines=200) + s(27, n_splines=200) + s(28, n_splines=200) + s(29, n_splines=200) + 
    s(30, n_splines=200) + s(31, n_splines=200) + s(32, n_splines=200) + s(33, n_splines=200) + s(34, n_splines=200) + 
    s(35, n_splines=200) + s(36, n_splines=200) + s(37, n_splines=200) + s(38, n_splines=200) + s(39, n_splines=200) + 
    s(40, n_splines=200) + s(41, n_splines=200) + s(42, n_splines=200) + s(43, n_splines=200) + s(44, n_splines=200) + 
    s(45, n_splines=200) + s(46, n_splines=200) + s(47, n_splines=200) + s(48, n_splines=200) + s(49, n_splines=200) + 
    s(50, n_splines=200) + s(51, n_splines=200) + s(52, n_splines=200) + s(53, n_splines=200) + s(54, n_splines=200) + 
    s(55, n_splines=200) + s(56, n_splines=200) + s(57, n_splines=200) + s(58, n_splines=200)
    , distribution='normal', link='identity')

GAM(callbacks=['deviance', 'diffs'], distribution='normal', 
   fit_intercept=True, link='identity', max_iter=100, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + s(11) + s(12) + s(13) + s(14) + s(15) + s(16) + s(17) + s(18) + s(19) + s(20) + s(21) + s(22) + s(23) + s(24) + s(25) + s(26) + s(27) + s(28) + s(29) + s(30) + s(31) + s(32) + s(33) + s(34) + s(35) + s(36) + s(37) + s(38) + s(39) + s(40) + s(41) + s(42) + s(43) + s(44) + s(45) + s(46) + s(47) + s(48) + s(49) + s(50) + s(51) + s(52) + s(53) + s(54) + s(55) + s(56) + s(57) + s(58),
   tol=0.0001, verbose=False)

In the next section, the model is fitted and because we are in the simplest setting, we can also use LinearGAM.

In [19]:
from pygam import LinearGAM, s

## model
gam = LinearGAM(s(0, n_splines=200) + s(1, n_splines=200) + s(2, n_splines=200) + s(3, n_splines=200) + s(4, n_splines=200) + 
    s(5, n_splines=200) + s(6, n_splines=200) + s(7, n_splines=200) + s(8, n_splines=200) + s(9, n_splines=200) +    
    s(10, n_splines=200) + s(11, n_splines=200) + s(12, n_splines=200) + s(13, n_splines=200) + s(14, n_splines=200) + 
    s(15, n_splines=200) + s(16, n_splines=200) + s(17, n_splines=200) + s(18, n_splines=200) + s(19, n_splines=200) + 
    s(20, n_splines=200) + s(21, n_splines=200) + s(22, n_splines=200) + s(23, n_splines=200) + s(24, n_splines=200) + 
    s(25, n_splines=200) + s(26, n_splines=200) + s(27, n_splines=200) + s(28, n_splines=200) + s(29, n_splines=200) + 
    s(30, n_splines=200) + s(31, n_splines=200) + s(32, n_splines=200) + s(33, n_splines=200) + s(34, n_splines=200) + 
    s(35, n_splines=200) + s(36, n_splines=200) + s(37, n_splines=200) + s(38, n_splines=200) + s(39, n_splines=200) + 
    s(40, n_splines=200) + s(41, n_splines=200) + s(42, n_splines=200) + s(43, n_splines=200) + s(44, n_splines=200) + 
    s(45, n_splines=200) + s(46, n_splines=200) + s(47, n_splines=200) + s(48, n_splines=200) + s(49, n_splines=200) + 
    s(50, n_splines=200) + s(51, n_splines=200) + s(52, n_splines=200) + s(53, n_splines=200) + s(54, n_splines=200) + 
    s(55, n_splines=200) + s(56, n_splines=200) + s(57, n_splines=200) + s(58, n_splines=200))
gam.fit(X_train, y_train)

LinearGAM(callbacks=[Deviance(), Diffs()], fit_intercept=True, 
   max_iter=100, scale=None, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + s(11) + s(12) + s(13) + s(14) + s(15) + s(16) + s(17) + s(18) + s(19) + s(20) + s(21) + s(22) + s(23) + s(24) + s(25) + s(26) + s(27) + s(28) + s(29) + s(30) + s(31) + s(32) + s(33) + s(34) + s(35) + s(36) + s(37) + s(38) + s(39) + s(40) + s(41) + s(42) + s(43) + s(44) + s(45) + s(46) + s(47) + s(48) + s(49) + s(50) + s(51) + s(52) + s(53) + s(54) + s(55) + s(56) + s(57) + s(58) + intercept,
   tol=0.0001, verbose=False)

In [20]:
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                    678.5999
Link Function:                     IdentityLink Log Likelihood:                                 -57459.876
Number of Samples:                         4000 AIC:                                           116278.9518
                                                AICc:                                          116557.6381
                                                GCV:                                           987027.8982
                                                Scale:                                         691093.1355
                                                Pseudo R-Squared:                                   0.7008
Feature Function                  Lam

<ipython-input-20-dec6a6acdaaa>:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  gam.summary()


Now crossvalidation has to be done to optimaze the penalization. For this, a grid search is carried out. In this point, the algorithm does not converge. But I will continue the next steps that were planned.

In [22]:
from pygam import LinearGAM, s

## model
gam = LinearGAM(s(0, n_splines=200) + s(1, n_splines=200) + s(2, n_splines=200) + s(3, n_splines=200) + s(4, n_splines=200) + 
    s(5, n_splines=200) + s(6, n_splines=200) + s(7, n_splines=200) + s(8, n_splines=200) + s(9, n_splines=200) +     
    s(10, n_splines=200) + s(11, n_splines=200) + s(12, n_splines=200) + s(13, n_splines=200) + s(14, n_splines=200) + 
    s(15, n_splines=200) + s(16, n_splines=200) + s(17, n_splines=200) + s(18, n_splines=200) + s(19, n_splines=200) + 
    s(20, n_splines=200) + s(21, n_splines=200) + s(22, n_splines=200) + s(23, n_splines=200) + s(24, n_splines=200) + 
    s(25, n_splines=200) + s(26, n_splines=200) + s(27, n_splines=200) + s(28, n_splines=200) + s(29, n_splines=200) + 
    s(30, n_splines=200) + s(31, n_splines=200) + s(32, n_splines=200) + s(33, n_splines=200) + s(34, n_splines=200) + 
    s(35, n_splines=200) + s(36, n_splines=200) + s(37, n_splines=200) + s(38, n_splines=200) + s(39, n_splines=200) + 
    s(40, n_splines=200) + s(41, n_splines=200) + s(42, n_splines=200) + s(43, n_splines=200) + s(44, n_splines=200) + 
    s(45, n_splines=200) + s(46, n_splines=200) + s(47, n_splines=200) + s(48, n_splines=200) + s(49, n_splines=200) + 
    s(50, n_splines=200) + s(51, n_splines=200) + s(52, n_splines=200) + s(53, n_splines=200) + s(54, n_splines=200) + 
    s(55, n_splines=200) + s(56, n_splines=200) + s(57, n_splines=200) + s(58, n_splines=200))
gam.gridsearch(np.array(X_train), y_train)

 18% (2 of 11) |####                     | Elapsed Time: 0:40:52 ETA:   3:53:27C:\Users\mhinojosalee\Anaconda3\lib\site-packages\pygam\pygam.py:752: RuntimeWarning: divide by zero encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
C:\Users\mhinojosalee\Anaconda3\lib\site-packages\pygam\pygam.py:752: RuntimeWarning: overflow encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
 27% (3 of 11) |######                   | Elapsed Time: 1:22:45 ETA:   5:35:03C:\Users\mhinojosalee\Anaconda3\lib\site-packages\pygam\pygam.py:752: RuntimeWarning: divide by zero encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
C:\Users\mhinojosalee\Anaconda3\lib\site-packages\pygam\pygam.py:752: RuntimeWarning: overflow encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
 36% (4 of 11) |#########                | Elapsed Time: 2:04:40 ETA:   4:53:31C:\Users\mhinojosalee\A

LinAlgError: SVD did not converge

We obtaine a new lambda and see the R2 of the program.

In [ ]:
from sklearn.metrics import r2_score
print('R2 : %.3f' % r2_score(gam.predict(np.array(X_train)), y_train))
print('R2 : %.3f' % r2_score(gam.predict(np.array(X_test)), y_test))

We pickle the model, to not lose it (it took 2.5 hours to run).

In [ ]:
score.to_pickle(data_output_path / "score_v3_GAM.pkl")
data_train.to_pickle(data_output_path / "data_train_v3_GAM.pkl")
joblib.dump(gbm_random, 'GAM_Q2_copy.pkl')

In [ ]:
plt.figure()
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    plt.plot(gam.partial_dependence(term=i))


In [ ]:
plt.figure()
plt.plot(gam.partial_dependence(term=3))

After this, we smooth the GAM. We use less basis functions (20 instead of 200)

In [ ]:
from pygam import LinearGAM, s

## model
gam_smooth = LinearGAM(s(0, n_splines=20) + s(1, n_splines=20) + s(2, n_splines=20) + s(3, n_splines=20) + s(4, n_splines=20) + 
    s(5, n_splines=20) + s(6, n_splines=20) + s(7, n_splines=20) + s(8, n_splines=20) + s(9, n_splines=20) +     
    s(10, n_splines=20) + s(11, n_splines=20) + s(12, n_splines=20) + s(13, n_splines=20) + s(14, n_splines=20) + 
    s(15, n_splines=20) + s(16, n_splines=20) + s(17, n_splines=20) + s(18, n_splines=20) + s(19, n_splines=20) + 
    s(20, n_splines=20) + s(21, n_splines=20) + s(22, n_splines=20) + s(23, n_splines=20) + s(24, n_splines=20) + 
    s(25, n_splines=20) + s(26, n_splines=20) + s(27, n_splines=20) + s(28, n_splines=20) + s(29, n_splines=20) + 
    s(30, n_splines=20) + s(31, n_splines=20) + s(32, n_splines=20) + s(33, n_splines=20) + s(34, n_splines=20) + 
    s(35, n_splines=20) + s(36, n_splines=20) + s(37, n_splines=20) + s(38, n_splines=20) + s(39, n_splines=20) + 
    s(40, n_splines=20) + s(41, n_splines=20) + s(42, n_splines=20) + s(43, n_splines=20) + s(44, n_splines=20) + 
    s(45, n_splines=20) + s(46, n_splines=20) + s(47, n_splines=20) + s(48, n_splines=20) + s(49, n_splines=20) + 
    s(50, n_splines=20) + s(51, n_splines=20) + s(52, n_splines=20) + s(53, n_splines=20) + s(54, n_splines=20) + 
    s(55, n_splines=20) + s(56, n_splines=20) + s(57, n_splines=20) + s(58, n_splines=20))
gam_smooth.gridsearch(np.array(X_train), y_train)


In [ ]:
plt.figure()
plt.plot(gam_smooth.partial_dependence(term=3))

In [ ]:
from sklearn.metrics import r2_score
print('R2 : %.3f' % r2_score(gam_smooth.predict(np.array(X_train)), y_train))
print('R2 : %.3f' % r2_score(gam_smooth.predict(np.array(X_test)), y_test))

In [ ]:
plt.figure()
XX = gam.generate_X_grid(term=3)
plt.plot(gam_smooth.partial_dependence(term=3, X=XX, width=.95)[1], c='r', ls='--')
plt.plot(gam_smooth.partial_dependence(term=3, X=XX), c='b')

In [ ]:
profit_preds_gam = gam.predict(score)

In [ ]:
np.save(data_output_path / "profit_preds_v3_GAM.pkl", profit_preds_gam)